In [1]:
import json
import pandas as pd
from pandas import json_normalize

In [2]:
# Загружаем данные
train = pd.read_json('train.json', orient='index').reset_index()
train.shape

(127755, 3)

In [21]:
train.sample()

,index,target,orders,site-meta,visits,last-visits-in-categories,exchange-sessions
19079,user_19080,male,"[{'site-id': 2, 'orders': [{'created-at': 1692...","[{'site-id': 2}, {'site-id': 30}]","[{'site-id': 3, 'first-seen': 1698167132, 'las...","[{'category': 'bank', 'last-visit-at': 1698166...",NaN


In [20]:
# Проверим сбалансированность
train['target'].value_counts(normalize=True)

target
female    0.50002
male      0.49998
Name: proportion, dtype: float64

In [4]:
# Развернем фичи
train = pd.concat([train.drop('features', axis = 1), pd.json_normalize(train['features'])], axis = 1)
train.shape

(127755, 7)

In [5]:
train.sample(3)

,index,target,orders,site-meta,visits,last-visits-in-categories,exchange-sessions
93189,user_93190,male,"[{'site-id': 478, 'orders': [{'created-at': 16...","[{'site-id': 109}, {'site-id': 355}, {'site-id...","[{'site-id': 2, 'first-seen': 1694066940, 'las...","[{'category': 'other', 'last-visit-at': 169808...",NaN
38812,user_38813,female,"[{'site-id': 8, 'orders': [{'created-at': 1687...","[{'site-id': 397}, {'site-id': 44, 'recency': ...","[{'site-id': 371, 'first-seen': 1694916651, 'l...","[{'category': 'sport', 'last-visit-at': 162964...","[{'landed-at': 1689055385, 'sites': [158, 1, 2..."
105959,user_105960,male,"[{'site-id': 619, 'orders': [{'created-at': 16...","[{'site-id': 619, 'recency': 2, 'frequency': 2...","[{'site-id': 371, 'first-seen': 1689093635, 'l...","[{'category': 'furniture', 'last-visit-at': 16...","[{'landed-at': 1692609352, 'sites': [46, 29, 1..."


In [40]:
# Проверим заполненность
print("Доля пропусков в train['features']:")
train.drop(columns=['index', 'target']).isna().mean()

Доля пропусков в train['features']:


orders                       0.095855
site-meta                    0.000016
visits                       0.013111
last-visits-in-categories    0.012164
exchange-sessions            0.413949
dtype: float64

In [9]:
# Последовательно точечно пройдем по структуре и посмотрим, что внутри
pd.json_normalize(train['orders'][0])

,site-id,orders
0,1,"[{'created-at': 1634292444, 'items': []}]"
1,2,"[{'created-at': 1639827297, 'items': []}, {'cr..."


In [69]:
w1 = pd.json_normalize(train['orders'][5])
pd.json_normalize(w1['orders'][0])

,created-at,items
0,1630352618,"[{'id': 'item_75', 'count': 1, 'general-catego..."
1,1648999970,"[{'id': 'item_78', 'count': 1, 'general-catego..."
2,1653315536,"[{'id': 'item_80', 'count': 1, 'general-catego..."


In [70]:
w10 = pd.json_normalize(w1['orders'][0])
pd.json_normalize(w10['items'][0])

,id,count,general-category-path,brand-id
0,item_75,1,"[7812065, 7812006, 7811879, 7877999]",2458
1,item_76,1,"[7811945, 7811896, 7811873, 7877999]",2458
2,item_77,1,"[7811945, 7811896, 7811873, 7877999]",2458


In [28]:
pd.json_normalize(train['site-meta'][10])

,site-id,recency,frequency,monetary
0,389,1.0,1.0,3.0
1,479,NaN,NaN,NaN
2,17,NaN,NaN,NaN
3,320,NaN,NaN,NaN
4,88,NaN,NaN,NaN


In [31]:
pd.json_normalize(train['visits'][10])

,site-id,first-seen,last-seen,visits
0,191,1692534275,1692534275,"[{'visited-at': 1692534275, 'session-duration'..."
1,3,1690569268,1697188144,"[{'visited-at': 1690569268, 'session-duration'..."
2,320,1690631283,1697462503,"[{'visited-at': 1690631283, 'session-duration'..."
3,217,1695130721,1695130721,"[{'visited-at': 1695130721, 'session-duration'..."
4,478,1688469761,1688469761,"[{'visited-at': 1688469761, 'session-duration'..."


In [65]:
pd.json_normalize(pd.json_normalize(train['visits'][10])['visits'][1])

,visited-at,session-duration,pages-count
0,1690569268,143,3
1,1690631190,55,2
2,1695130521,52,2
3,1697188144,58,2


In [60]:
pd.json_normalize(train['exchange-sessions'][5])

,landed-at,sites,clicks,accepted-site-id,accepted-at
0,1698263812,"[169, 214, 215, 216, 217, 218, 1, 219, 220, 22...","[{'clicked-at': 1698263944, 'site-id': 207}]",207.0,1.698264e+09
1,1658944504,"[46, 21, 29, 78, 180, 158, 42, 74, 1, 119, 22,...","[{'clicked-at': 1658944723, 'site-id': 1}]",1.0,1.658945e+09
2,1646471462,"[326, 71, 2, 1, 21, 171, 31, 74, 80, 325, 335,...",NaN,NaN,NaN
3,1644170436,"[326, 71, 2, 1, 21, 48, 336, 74, 75, 79, 92, 8...","[{'clicked-at': 1644170563, 'site-id': 21}]",NaN,NaN
4,1630352621,"[21, 31, 74, 83, 90, 146, 88, 98, 80, 180, 46,...","[{'clicked-at': 1630352688, 'site-id': 21}]",21.0,1.630353e+09
5,1596455506,"[80, 180, 97, 151, 174, 159, 40, 81, 199, 21, ...","[{'clicked-at': 1596455528, 'site-id': 151}]",NaN,NaN
6,1578392090,"[34, 188, 179, 81, 174, 46, 175, 181, 208, 86,...","[{'clicked-at': 1578392170, 'site-id': 34}]",34.0,1.578392e+09
7,1509304461,"[37, 337, 184, 341, 5, 147, 342, 79, 93, 343, ...","[{'clicked-at': 1509304683, 'site-id': 16}, {'...",16.0,1.509305e+09


In [64]:
pd.json_normalize(pd.json_normalize(train['exchange-sessions'][5])['clicks'][7])

,clicked-at,site-id
0,1509304683,16
1,1509304764,16


In [43]:
pd.json_normalize(train['last-visits-in-categories'][2])

,category,last-visit-at
0,electronics,1686982782
1,other,1696320204
